In [20]:
import gensim
from gensim.models import FastText,fasttext
import pandas as pd
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import math
import re
from string import digits

In [2]:
qc_embeding_path = './embeddings/lematized_embedding_v1.model'
qc_embeddings = FastText.load(qc_embeding_path)

In [3]:
es_embeding_path = './embeddings/cc.es.300.bin'
es_embeddings = fasttext.load_facebook_vectors(es_embeding_path)

In [14]:
es_embeddings.key_to_index

{'de': 0,
 ',': 1,
 '.': 2,
 'la': 3,
 'y': 4,
 'en': 5,
 'que': 6,
 'el': 7,
 '</s>': 8,
 'a': 9,
 'los': 10,
 ':': 11,
 '"': 12,
 'del': 13,
 'un': 14,
 ')': 15,
 'se': 16,
 'con': 17,
 'por': 18,
 'las': 19,
 '(': 20,
 'para': 21,
 'una': 22,
 'es': 23,
 'no': 24,
 'su': 25,
 'al': 26,
 'como': 27,
 'lo': 28,
 '/': 29,
 'más': 30,
 'El': 31,
 'o': 32,
 "'": 33,
 'La': 34,
 '!': 35,
 '|': 36,
 '?': 37,
 'me': 38,
 'En': 39,
 '...': 40,
 '-': 41,
 'sus': 42,
 'este': 43,
 'pero': 44,
 'ha': 45,
 'esta': 46,
 ';': 47,
 '“': 48,
 '_': 49,
 '”': 50,
 'si': 51,
 'sobre': 52,
 '¿': 53,
 'fue': 54,
 'son': 55,
 'le': 56,
 'muy': 57,
 'ser': 58,
 'ya': 59,
 'tu': 60,
 'todo': 61,
 '1': 62,
 'entre': 63,
 'te': 64,
 'mi': 65,
 'Los': 66,
 '%': 67,
 'sin': 68,
 'también': 69,
 'está': 70,
 'desde': 71,
 'años': 72,
 'puede': 73,
 'dos': 74,
 '2': 75,
 'cuando': 76,
 'hasta': 77,
 'nos': 78,
 'tiene': 79,
 'todos': 80,
 'A': 81,
 'No': 82,
 'Y': 83,
 'donde': 84,
 'parte': 85,
 '3': 86,
 'hay':

In [62]:
dataset = pd.DataFrame(columns=['quechua', 'spanish'])
dataset['quechua'] = pd.read_csv('./train_data/QuechuaCollaoCorpus.csv')['Quechua']
dataset['spanish'] = pd.read_csv('./train_data/QuechuaCollaoCorpus.csv')['Traducción']
dataset

,quechua,spanish
0,Ch’arwi,Desorden
1,Runa,Persona
2,Chanin,Precio
3,Puka,Rojo
4,Hallp’a,Tierra
...,...,...
2066,Runakuna manan mallkikunata tarpunkuchu chaymi...,La personas no plantan árboles es uno de los f...
2067,Erqe mamanpa wañusqanmanta waqan.,El niño llora porque su madre murió.
2068,Huk wayna chakamanta wukchuyukusqa hinaspa wañ...,Un joven se lanzó del puente y murió.
2069,Mama churin mana wasinman kutimusqanmanta waqan.,La madre llora porque su hijo no vuelve a la c...


In [38]:
dataset.isna().sum()

quechua    2
spanish    1
dtype: int64

In [63]:
dataset.dropna(inplace=True)
dataset.reset_index(drop=True, inplace=True)

In [64]:

def preprocess_sentence(sentence):
    num_digits = str.maketrans('', '', digits)

    sentence = sentence.lower()
    sentence = re.sub(" +", " ", sentence)
    sentence = re.sub(" . ", "", sentence)
    sentence = sentence.translate(num_digits)
    sentence = sentence.strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = sentence.rstrip().strip()
    
    # Remove the final dot if it exists
    if sentence.endswith('.'):
        sentence = sentence[:-1].strip()
    
    sentence = 'start_ ' + sentence + ' _end'  # importante

    return sentence

def clean_df(df):
    df['quechua'] = df['quechua'].apply(lambda x: preprocess_sentence(x))
    df['spanish'] = df['spanish'].apply(lambda x: preprocess_sentence(x))

In [65]:
clean_df(dataset)

In [66]:
dataset

,quechua,spanish
0,start_ ch’arwi _end,start_ desorden _end
1,start_ runa _end,start_ persona _end
2,start_ chanin _end,start_ precio _end
3,start_ puka _end,start_ rojo _end
4,start_ hallp’a _end,start_ tierra _end
...,...,...
2064,start_ runakuna manan mallkikunata tarpunkuchu...,start_ la personas no plantan árboles es uno d...
2065,start_ erqe mamanpa wañusqanmanta waqan _end,start_ el niño llora porque su madre murió _end
2066,start_ huk wayna chakamanta wukchuyukusqa hina...,start_ un joven se lanzó del puentemurió _end
2067,start_ mama churin mana wasinman kutimusqanman...,start_ la madre llora porque su hijo no vuelve...


In [6]:
qc.build_vocab(dataset['quechua'], update=True)
embeddings.train(dataset['quechua'], total_examples=embeddings.corpus_count, epochs=15,)

TypeError: 'float' object is not iterable

In [7]:
class PositionalEncoding(nn.Module):
    r"""Inject some information about the relative or absolute position of the tokens in the sequence.
        The positional encodings have the same dimension as the embeddings, so that the two can be summed.
        Here, we use sine and cosine functions of different frequencies.
    .. math:
        \text{PosEncoder}(pos, 2i) = sin(pos/10000^(2i/d_model))
        \text{PosEncoder}(pos, 2i+1) = cos(pos/10000^(2i/d_model))
        \text{where pos is the word position and i is the embed idx)
    Args:
        d_model: the embed dim (required).
        dropout: the dropout value (default=0.1).
        max_len: the max. length of the incoming sequence (default=5000).
    Examples:
        >>> pos_encoder = PositionalEncoding(d_model)
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        r"""Inputs of forward function
        Args:
            x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        Examples:
            >>> output = pos_encoder(x)
        """

        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [8]:
class TransformerModel(nn.Transformer):
    """Container module with an encoder, a recurrent or transformer module, and a decoder."""

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__(d_model=ninp, nhead=nhead, dim_feedforward=nhid, num_encoder_layers=nlayers)
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)

        self.input_emb = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        return torch.log(torch.tril(torch.ones(sz,sz)))

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.input_emb.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src, has_mask=True):
        if has_mask:
            device = src.device
            if self.src_mask is None or self.src_mask.size(0) != len(src):
                mask = self._generate_square_subsequent_mask(len(src)).to(device)
                self.src_mask = mask
        else:
            self.src_mask = None

        src = self.input_emb(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.encoder(src, mask=self.src_mask)
        output = self.decoder(output)
        return F.log_softmax(output, dim=-1)

In [35]:
out.size()

torch.Size([20, 32, 512])

In [10]:
#pass embeding fasttext to embedding layer

vectors  = torch.tensor(embeddings.wv.vectors)

In [15]:
vectors.size()

torch.Size([2561, 50])

In [12]:
weight = torch.FloatTensor(vectors)
embedding = nn.Embedding.from_pretrained(weight)

In [27]:
embeddings.wv['wasi']

array([-0.50889   ,  0.1686052 , -0.24421082,  0.15753326, -0.36160132,
        0.24685092, -0.18002182, -0.17252123, -0.28220466,  0.38489878,
       -0.02020847,  0.21663795,  0.05879577, -0.15637793,  0.24656545,
       -0.24084774,  0.07427774,  0.5354595 ,  0.253039  , -0.18267351,
        0.30777204,  0.32439196,  0.2406271 , -0.3227719 , -0.404154  ,
       -0.41446486, -0.19660893,  0.13176258, -0.5842336 , -0.5161695 ,
        0.72205323,  0.07310987,  0.3700086 , -0.09453848,  0.10423267,
       -0.1513904 ,  0.5310659 ,  0.1339472 , -0.16158412, -0.44667733,
       -0.12508145,  0.09292982, -0.06693438,  0.01572501,  0.17116961,
       -0.12222562,  0.12246072,  0.12925214, -0.03877937, -0.1251332 ],
      dtype=float32)

In [21]:
embedding(torch.LongTensor([0]))

tensor([[-0.3116,  0.3042,  0.1542,  0.0290,  0.4287, -0.3780, -0.3416,  0.0329,
         -0.3269,  0.1054, -0.0190,  0.2326, -0.0456,  0.2658, -0.1921,  0.3213,
          0.3945,  0.3484,  0.2451, -0.3252, -0.2175,  0.2806,  0.1836, -0.1274,
         -0.2662, -0.5361, -0.5020, -0.0786,  0.1853, -0.5092,  0.6678,  0.0465,
         -0.3356,  0.2425,  0.1637, -0.1459, -0.2762, -0.0944,  0.0261,  0.0998,
         -0.0642, -0.3979,  0.2708,  0.5895, -0.0360,  0.5141,  0.0692,  0.2658,
          0.5971,  0.0239]])